## Pandas Dataframe to GeoTIFF Manipulation

In [1]:
import ee
from osgeo import gdal
from osgeo import osr
import pandas as pd
import numpy as np
import time
import requests

ee.Initialize()

### The input dataframe refers to Fundão dam (Mariana-MG-Brazil)

In [2]:
tab1 = pd.read_csv(r'C:\Users\vinim\IC_20_21\Mariana_NDWI.csv')
tabNDWI = tab1.set_index(['Latitude','Longitude'])
t1time = tabNDWI.iloc[:,:-4] #datetime columns
t1time.columns = pd.to_datetime(t1time.columns, format='%d-%m-%Y')
t1time = t1time.reindex(sorted(t1time.columns), axis=1)
dataframes = [t1time, tabNDWI.iloc[:,-4:]]
resNDWI = pd.concat(dataframes, axis=1)

In [3]:
tab = pd.read_csv(r'C:\Users\vinim\IC_20_21\Mariana_anomaly.csv') #CSV file from anomaly detections algorithms
tabin = tab.set_index(['Latitude','Longitude'])
dftime = tabin.iloc[:,:-6] #datetime columns
dftime.columns = pd.to_datetime(dftime.columns, format='%d-%m-%Y')
dftime = dftime.reindex(sorted(dftime.columns), axis=1)
dataframes = [dftime, tabin.iloc[:,-6:]]
resNDVI = pd.concat(dataframes, axis=1)
resNDVI = resNDVI.set_index(resNDWI.index)

### Anomaly detection methods used
- Isolation Forest
- One-Class SVM
- DBSCAN
- K-Means

In [4]:
pd.options.mode.chained_assignment = None
import sklearn
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

def count_kmeans(kmean):
    return np.count_nonzero(kmean)

def count_anomaly(predict):
    return np.count_nonzero(predict==-1)

def OCSVM(df):
    scaler = StandardScaler()
    y = df.to_numpy().reshape(-1, 1)
    y_s = scaler.fit_transform(y)
    res = OneClassSVM(nu=0.05).fit_predict(y_s)
    return count_anomaly(res)

def ISO(df):
    scaler = StandardScaler()
    y = df.to_numpy().reshape(-1, 1)
    y_s = scaler.fit_transform(y)
    res = IsolationForest(n_estimators=5,random_state=0).fit_predict(y_s)
    return count_anomaly(res)

def kmeans(df):
    scaler = StandardScaler()
    X = df.to_numpy().reshape(-1,1)
    X_scaled = scaler.fit_transform(X)
    km_clust = KMeans(n_clusters=2)
    assigned_clusters = km_clust.fit_predict(X_scaled)
    return count_kmeans(assigned_clusters)

def DS(df):
    scaler = StandardScaler()
    y = df.to_numpy().reshape(-1, 1)
    y_s = scaler.fit_transform(y)
    outlier_detection = DBSCAN(eps = .2, metric='euclidean', min_samples = 5, n_jobs = -1)
    clusters = outlier_detection.fit_predict(y_s)
    return count_kmeans(clusters)

### Dataframe to TIFF Function
- Each meathod was saved as a Raster band
- The columns names must to be passed as list input (in our case, the AD methods)

In [5]:
def save_tiff_fromdf(df,bands,dummy,path_out):
    
    lat = []
    lon = []
    for i in range(len(df)):
        lat.append(df.index[i][0])
        lon.append(df.index[i][1])
    
    
    ulat = np.unique(lat)
    ulon = np.unique(lon)
    ncols = len(ulon)
    nrows = len(ulat)
    nbands = len(bands)
    ys = ulat[11]-ulat[10]
    xs = ulon[11]-ulon[10]
    
    arr = np.zeros([nbands, nrows, ncols], np.float32)
    refLat = np.max(ulat)
    refLon = np.min(ulon)
    for j in range(len(df)):
        posLin = np.int64( np.round( (refLat - lat[j])/ys ) )
        posCol = np.int64( np.round( (lon[j] - refLon)/xs ) )
        for b in range(nbands):
            arr[b,posLin,posCol] = df.loc[df.index[j],bands[b]]
            
    transform = (np.min(ulon),xs,0,np.max(ulat),0,-ys)
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)
    
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(path_out,ncols,nrows,nbands,gdal.GDT_Float32)
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(target.ExportToWkt())

    ind = 1
    for b in range(nbands):
        bandArr = np.copy(arr[b,:,:])
        outBand = outDs.GetRasterBand(ind)
        outBand.WriteArray(bandArr)
        outBand.FlushCache()
        outBand.SetNoDataValue(dummy)
        ind += 1

    outDs = None
    del outDs, outBand

    return 'ok...'

At this point, we have to change the index of NDVI Dataset because a index rounding occurred during file compression

In [6]:
bandlist = ['One-Class SVM','Isolation Forest','K-Means','DBSCAN']
dummy = -99999

### The outputs from cells below are available at repository

In [ ]:
path_out = 'M_NDWIanomaly.tif'
save_tiff_fromdf(resNDWI,bandlist,dummy,path_out)

In [ ]:
path_out = 'M_NDVIanomaly.tif'
save_tiff_fromdf(resNDVI,bandlist,dummy,path_out)